##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Quantum advantage in learning from experiments

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/quantum/tutorials/quantum_advantage_in_learning_from_experiments"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/quantum/blob/master/docs/tutorials/quantum_advantage_in_learning_from_experiments.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/quantum/blob/master/docs/tutorials/quantum_advantage_in_learning_from_experiments.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/quantum/docs/tutorials/quantum_advantage_in_learning_from_experiments.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This tutorial shows the experiments of <a target="_blank" href="https://arxiv.org/abs/2112.00778" class="external">Quantum advantage in learning from experiments</a>.

## Setup

In [ ]:
!pip install tensorflow==2.4.1

Install TensorFlow Quantum:

In [ ]:
!pip install tensorflow-quantum

In [ ]:
# Update package resources to account for version changes.
import importlib, pkg_resources
importlib.reload(pkg_resources)

Now import TensorFlow and the module dependencies:

In [1]:
import tensorflow as tf
import cirq
from cirq_google import SycamoreGate
import numpy as np
import sympy
import functools
import os

## 1. The Basics

In [2]:
def create_randomized_sweeps(
    hidden_p,
    symbols,
    n_params,
    rand_state):
    last_i = 0
    for i, pauli in enumerate(hidden_p):
        if pauli != "I":
            last_i = i

    sign_p = rand_state.choice([1, -1])
    all_sweeps = []
    for _ in range(n_params):
        current_sweep = dict()
        for twocopy in [0, 1]:
            parity = sign_p * rand_state.choice([1, -1], p=[0.95, 0.05])
            for i, pauli in enumerate(hidden_p):
                current_symbol = symbols[2 * i + twocopy]
                current_sweep[current_symbol] = rand_state.choice([0, 1])
                if pauli != "I":
                    if last_i == i:
                        v = 1 if parity == -1 else 0
                        current_sweep[current_symbol] = v
                    elif current_sweep[current_symbol] == 1:
                        parity *= -1

        all_sweeps.append(current_sweep)
    return all_sweeps



def un_bell_pair_block(qubits):
    return [cirq.CNOT(qubits[0], qubits[1]), cirq.H(qubits[0])]

def inv_z_basis_gate(pauli):
    """Returns inverse Z basis transformation ops for a given Pauli.

    Args:
        pauli: Python str representing a single pauli.

    Returns:
        Corresponding `cirq.Gate` to do the inverse basis conversion.
    """
    if pauli == "I" or pauli == "Z":
        return cirq.I
    if pauli == "X":
        return cirq.H
    if pauli == "Y":
        # S^dag H to get to computational, H S to go back.
        return cirq.PhasedXZGate(
            axis_phase_exponent=-0.5, x_exponent=0.5, z_exponent=-0.5
        )
    raise ValueError("Invalid Pauli.")
    
def build_circuit(
    qubit_pairs,
    pauli,
    n_shots,
    rand_state):
    """Create I + P problem circuit between qubit pairs.

    Args:
        qubit_pairs: List of qubit pairs.
        pauli: Python str containing characters 'I', 'X', 'Y' or 'Z'.
        n_shots: Number of repetitions to generate for sweeps.

    Returns:
        A (circuit, sweep) tuple, runnable using `run_sweep`.
    """
    a_qubits = [pair[0] for pair in qubit_pairs]
    b_qubits = [pair[1] for pair in qubit_pairs]
    all_qubits = np.concatenate(qubit_pairs)

    flip_params = sympy.symbols(f"param_0:{len(qubit_pairs) * 2}")

    # Add X flips.
    ret_circuit = cirq.Circuit(cirq.X(q) ** p for q, p in zip(all_qubits, flip_params))

    # Add basis turns a and b.
    ret_circuit += [
        inv_z_basis_gate(p)(q) for q, p in zip(a_qubits, pauli)
    ]
    ret_circuit += [
        inv_z_basis_gate(p)(q) for q, p in zip(b_qubits, pauli)
    ]

    # Add un-bell pair.
    ret_circuit += [un_bell_pair_block(pair) for pair in qubit_pairs]

    # Add measurements.
    for i, qubit in enumerate(all_qubits):
        ret_circuit += cirq.measure(qubit, key=f"q{i}")

    # Create randomized flippings. These flippings will contain values of 1,0.
    # which will turn the X gates on or off.
    params = create_randomized_sweeps(
        pauli, flip_params, n_shots, rand_state
    )
    return ret_circuit, params



In [28]:
rand_source = np.random.RandomState(20160913)
n_paulis = 200
n = 3
n_shots = 47
n_sweeps = 1

paulis = np.array(["X", "Y", "Z", "I"])
pauli_strings = rand_source.choice(a=paulis, size=(n_paulis, n), replace=True)

system_pairs = [(cirq.GridQubit(0, i), cirq.GridQubit(1, i)) for i in range(n)]

simulator = cirq.Simulator()

all_results = []
for pauli in pauli_strings:
    circuit, sweeps = build_circuit(system_pairs, pauli, n_shots, rand_source)
    
    results_for_pauli = []
    for b in range(0, n_shots, n_sweeps):
        results = simulator.run_sweep(
            program=circuit,
            params=sweeps[b : b + n_sweeps],
            repetitions=1
        )

        batch_results = []
        for j, single_circuit_samples in enumerate(results):
            qubit_order = [f"q{i}" for i in range(2 * n)]
            out0 = single_circuit_samples.data[qubit_order].to_numpy()
            batch_results.append(np.squeeze(out0))

        batch_results = np.array(batch_results)
        results_for_pauli.append(batch_results)
    all_results.append(np.concatenate(results_for_pauli))
    

In [29]:
def _encode_pauli(paulis):
    encoded = []
    for pauli in paulis:
        if pauli == 'I':
            encoded.extend([0, 0])
        elif pauli == 'X':
            encoded.extend([0, 1])
        elif pauli == 'Y':
            encoded.extend([1, 0])
        elif pauli == 'Z':
            encoded.extend([1, 1])
    return np.asarray([encoded])
    
inputs = []
targets = []
    
for i in range(n_paulis):
    encoded_pauli = np.repeat(_encode_pauli(pauli_strings[i, :]), n_shots, axis=0)
    
    inputs.append(np.expand_dims(np.concatenate((all_results[i], encoded_pauli,), axis=1), axis=0))
    targets.append([1, 0])
    
    inputs.append(np.expand_dims(np.concatenate((all_results[(i + 1) % len(all_results)], encoded_pauli,), axis=1), axis=0))
    targets.append([0, 1])

inputs = np.concatenate(inputs)
targets = np.asarray(targets)

inputs.shape, targets.shape


((400, 47, 12), (400, 2))

In [31]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(8, activation='relu', use_bias=True))
model.add(tf.keras.layers.GRU(4, go_backwards=False, return_sequences=True))
model.add(tf.keras.layers.GRU(4, go_backwards=True,  return_sequences=True))
model.add(tf.keras.layers.GRU(4,  go_backwards=False))
model.add(tf.keras.layers.Dense(8, activation='relu', use_bias=True))
model.add(tf.keras.layers.Dense(2, activation='softmax', use_bias=True))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

history = model.fit(
    x=inputs.astype(float),
    y=targets.astype(float),
    epochs=500,
    verbose=1)

print(x.shape)
print(y.shape)
print(model.output_shape)
print(history.history['accuracy'][0])
print(history.history['loss'][0])
print(history.history['accuracy'][-1])
print(history.history['loss'][-1])


Epoch 1/500
13/13 [==============================] - 6s 62ms/step - loss: 0.6949 - accuracy: 0.4975
Epoch 2/500
13/13 [==============================] - 1s 63ms/step - loss: 0.6941 - accuracy: 0.4850
Epoch 3/500
13/13 [==============================] - 1s 62ms/step - loss: 0.6934 - accuracy: 0.4925
Epoch 4/500
13/13 [==============================] - 1s 62ms/step - loss: 0.6934 - accuracy: 0.4950
Epoch 5/500
13/13 [==============================] - 1s 62ms/step - loss: 0.6933 - accuracy: 0.5000
Epoch 6/500
13/13 [==============================] - 1s 62ms/step - loss: 0.6933 - accuracy: 0.5000
Epoch 7/500
13/13 [==============================] - 1s 63ms/step - loss: 0.6932 - accuracy: 0.4900
Epoch 8/500
13/13 [==============================] - 1s 62ms/step - loss: 0.6932 - accuracy: 0.5275
Epoch 9/500
13/13 [==============================] - 1s 62ms/step - loss: 0.6933 - accuracy: 0.4975
Epoch 10/500
13/13 [==============================] - 1s 62ms/step - loss: 0.6933 - accuracy: 0.5100

13/13 [==============================] - 1s 62ms/step - loss: 0.5758 - accuracy: 0.6700
Epoch 83/500
13/13 [==============================] - 1s 63ms/step - loss: 0.5697 - accuracy: 0.6725
Epoch 84/500
13/13 [==============================] - 1s 62ms/step - loss: 0.5627 - accuracy: 0.6750
Epoch 85/500
13/13 [==============================] - 1s 63ms/step - loss: 0.5544 - accuracy: 0.6675
Epoch 86/500
13/13 [==============================] - 1s 62ms/step - loss: 0.5626 - accuracy: 0.6700
Epoch 87/500
13/13 [==============================] - 1s 62ms/step - loss: 0.5595 - accuracy: 0.6625
Epoch 88/500
13/13 [==============================] - 1s 62ms/step - loss: 0.5611 - accuracy: 0.6800
Epoch 89/500
13/13 [==============================] - 1s 63ms/step - loss: 0.5542 - accuracy: 0.6925
Epoch 90/500
13/13 [==============================] - 1s 62ms/step - loss: 0.5661 - accuracy: 0.6700
Epoch 91/500
13/13 [==============================] - 1s 62ms/step - loss: 0.5587 - accuracy: 0.6725
Epo

13/13 [==============================] - 1s 62ms/step - loss: 0.3662 - accuracy: 0.8275
Epoch 163/500
13/13 [==============================] - 1s 62ms/step - loss: 0.3607 - accuracy: 0.8125
Epoch 164/500
13/13 [==============================] - 1s 62ms/step - loss: 0.3881 - accuracy: 0.8050
Epoch 165/500
13/13 [==============================] - 1s 62ms/step - loss: 0.4028 - accuracy: 0.7800
Epoch 166/500
13/13 [==============================] - 1s 61ms/step - loss: 0.5197 - accuracy: 0.7225
Epoch 167/500
13/13 [==============================] - 1s 62ms/step - loss: 0.5323 - accuracy: 0.7325
Epoch 168/500
13/13 [==============================] - 1s 62ms/step - loss: 0.3757 - accuracy: 0.7900
Epoch 169/500
13/13 [==============================] - 1s 62ms/step - loss: 0.3580 - accuracy: 0.8225
Epoch 170/500
13/13 [==============================] - 1s 62ms/step - loss: 0.3218 - accuracy: 0.8475
Epoch 171/500
13/13 [==============================] - 1s 62ms/step - loss: 0.2953 - accuracy: 0

13/13 [==============================] - 1s 62ms/step - loss: 0.1311 - accuracy: 0.9575
Epoch 243/500
13/13 [==============================] - 1s 63ms/step - loss: 0.1219 - accuracy: 0.9600
Epoch 244/500
13/13 [==============================] - 1s 62ms/step - loss: 0.1169 - accuracy: 0.9675
Epoch 245/500
13/13 [==============================] - 1s 62ms/step - loss: 0.1112 - accuracy: 0.9650
Epoch 246/500
13/13 [==============================] - 1s 62ms/step - loss: 0.1120 - accuracy: 0.9700
Epoch 247/500
13/13 [==============================] - 1s 62ms/step - loss: 0.1077 - accuracy: 0.9625
Epoch 248/500
13/13 [==============================] - 1s 62ms/step - loss: 0.1066 - accuracy: 0.9600
Epoch 249/500
13/13 [==============================] - 1s 62ms/step - loss: 0.1050 - accuracy: 0.9600
Epoch 250/500
13/13 [==============================] - 1s 62ms/step - loss: 0.0980 - accuracy: 0.9700
Epoch 251/500
13/13 [==============================] - 1s 62ms/step - loss: 0.1006 - accuracy: 0

13/13 [==============================] - 1s 62ms/step - loss: 0.2473 - accuracy: 0.9225
Epoch 323/500
13/13 [==============================] - 1s 64ms/step - loss: 0.2500 - accuracy: 0.8950
Epoch 324/500
13/13 [==============================] - 1s 64ms/step - loss: 0.2377 - accuracy: 0.8975
Epoch 325/500
13/13 [==============================] - 1s 63ms/step - loss: 0.1564 - accuracy: 0.9400
Epoch 326/500
13/13 [==============================] - 1s 64ms/step - loss: 0.1568 - accuracy: 0.9350
Epoch 327/500
13/13 [==============================] - 1s 63ms/step - loss: 0.1350 - accuracy: 0.9525
Epoch 328/500
13/13 [==============================] - 1s 62ms/step - loss: 0.1084 - accuracy: 0.9625
Epoch 329/500
13/13 [==============================] - 1s 62ms/step - loss: 0.0945 - accuracy: 0.9750
Epoch 330/500
13/13 [==============================] - 1s 62ms/step - loss: 0.0906 - accuracy: 0.9750
Epoch 331/500
13/13 [==============================] - 1s 62ms/step - loss: 0.0914 - accuracy: 0

13/13 [==============================] - 1s 62ms/step - loss: 0.5259 - accuracy: 0.8325
Epoch 403/500
13/13 [==============================] - 1s 62ms/step - loss: 0.3587 - accuracy: 0.8700
Epoch 404/500
13/13 [==============================] - 1s 62ms/step - loss: 0.2717 - accuracy: 0.9050
Epoch 405/500
13/13 [==============================] - 1s 62ms/step - loss: 0.2202 - accuracy: 0.9200
Epoch 406/500
13/13 [==============================] - 1s 62ms/step - loss: 0.2118 - accuracy: 0.9225
Epoch 407/500
13/13 [==============================] - 1s 62ms/step - loss: 0.2163 - accuracy: 0.9125
Epoch 408/500
13/13 [==============================] - 1s 62ms/step - loss: 0.1425 - accuracy: 0.9450
Epoch 409/500
13/13 [==============================] - 1s 62ms/step - loss: 0.1141 - accuracy: 0.9525
Epoch 410/500
13/13 [==============================] - 1s 62ms/step - loss: 0.0832 - accuracy: 0.9700
Epoch 411/500
13/13 [==============================] - 1s 62ms/step - loss: 0.0794 - accuracy: 0

13/13 [==============================] - 1s 61ms/step - loss: 0.3427 - accuracy: 0.9025
Epoch 483/500
13/13 [==============================] - 1s 63ms/step - loss: 0.9733 - accuracy: 0.7825
Epoch 484/500
13/13 [==============================] - 1s 63ms/step - loss: 0.8728 - accuracy: 0.7025
Epoch 485/500
13/13 [==============================] - 1s 63ms/step - loss: 0.7868 - accuracy: 0.7600
Epoch 486/500
13/13 [==============================] - 1s 62ms/step - loss: 0.8302 - accuracy: 0.7100
Epoch 487/500
13/13 [==============================] - 1s 62ms/step - loss: 0.7626 - accuracy: 0.7025
Epoch 488/500
13/13 [==============================] - 1s 62ms/step - loss: 0.6386 - accuracy: 0.7450
Epoch 489/500
13/13 [==============================] - 1s 62ms/step - loss: 0.4899 - accuracy: 0.8000
Epoch 490/500
13/13 [==============================] - 1s 62ms/step - loss: 0.4116 - accuracy: 0.8075
Epoch 491/500
13/13 [==============================] - 1s 62ms/step - loss: 0.4566 - accuracy: 0